In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
import numpy as np
import random as rd
import math
import datetime
from shutil import make_archive

from scipy.optimize.optimize import _prepare_scalar_function
from scipy.optimize.optimize import vecnorm
from scipy.optimize.optimize import _line_search_wolfe12
from scipy.optimize.optimize import _LineSearchError
from scipy.optimize.optimize import OptimizeResult
from scipy.optimize.optimize import _status_message
from scipy.optimize._differentiable_functions import ScalarFunction, FD_METHODS
from scipy.optimize.linesearch import (line_search_wolfe1, line_search_wolfe2,
                         line_search_wolfe2 as line_search,
                         LineSearchWarning)

15:26: E128 continuation line under-indented for visual indent
16:26: E128 continuation line under-indented for visual indent


In [4]:
class BFGS:
    """ Optimizer class for BFGS algorithm.

    Parameters:
    -----------------
    budget : int
        Budget for function evaluations.

    Attributes:
    -----------------
    d : int
        The number of dimensions, problem variables.

    gtol : float
        Value for gradient tolerance.

    norm : float

    eps : float

    return_all : bool, optional
        Set to True to return a list of the best solution at each of the
        iterations.

    finite_diff_rel_step : None or array_like, optional
        If `jac in ['2-point', '3-point', 'cs']` the relative step size to
        use for numerical approximation of the jacobian. The absolute step
        size is computed as ``h = rel_step * sign(x0) * max(1, abs(x0))``,
        possibly adjusted to fit into the bounds. For ``method='3-point'``
        the sign of `h` is ignored. If None (default) then step is selected
        automatically.

    """

    def __init__(self, budget):
        self.budget = budget
        self.d = 1
        self.gtol = 1e-10
        self.norm = np.inf
        self.eps = math.sqrt(np.finfo(float).eps)
        self.return_all = False
        self.jac = None
        self.finite_diff_rel_step = None

    def __call__(self, func):
        """ Runs the BFGS algorithm.

        Parameters:
        --------------
        func : object-like
            The function to be optimized.

        Returns:
        --------------
        x_opt : array
                The best found solution.

        f_opt: float
               The fitness value for the best found solution.

        """

        # Initialization
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        x_opt = None
        f_opt = np.inf
        retall = self.return_all
        I = np.eye(self.d, dtype=int)    # identity matrix
        Hk = np.eye(self.d, dtype=int)   # B0 = identity
        k = 0

        # Initialize first point x0 at random
        x0 = np.zeros(self.d)
        for i in range(0, self.d):
            x0[i] = rd.uniform(-5, 5)

        # Prepare scalar function object and derive function and gradient function
        sf = _prepare_scalar_function(func, x0, self.jac, epsilon=self.eps,
                              finite_diff_rel_step=self.finite_diff_rel_step)
        f = sf.fun    # function object to evaluate function
        gradient = sf.grad    # function object to evaluate gradient

        old_fval = f(x0)    # evaluate x0
        gfk = gradient(x0)   # gradient at x0

        # Sets the initial step guess to dx ~ 1
        old_old_fval = old_fval + np.linalg.norm(gfk) / 2

        xk = x0
        if retall:
            allvecs = [x0]

        # Calculate initial gradient norm
        gnorm = vecnorm(gfk, ord=self.norm)

        # Algorithm loop
        while (func.evaluations < eval_budget) and not func.best_so_far_precision <= 1e-8:
            pk = -np.dot(Hk, gfk)    # derive direction pk from HK and gradient at x0 (gfk)
            # Derive alpha_k with Wolfe conditions
            try:
                alpha_k, fc, gc, old_fval, old_old_fval, gfkp1 = \
                     _line_search_wolfe12(f, gradient, xk, pk, gfk,
                                          old_fval, old_old_fval, amin=1e-100, amax=1e100)
            except _LineSearchError:
                break

            # calculate xk+1 with alpha_k and pk
            xkp1 = xk + alpha_k * pk
            if retall:
                allvecs.append(xkp1)
            sk = xkp1 - xk    # step sk is difference between xk+1 and xk
            xk = xkp1    # make xk+1 new xk for next iteration
            # Calculate gradient of xk+1 if not already found by Wolfe search
            if gfkp1 is None:
                gfkp1 = gradient(xkp1)
            yk = gfkp1 - gfk    # gradient difference
            gfk = gfkp1    # copy gradient to gfk for new iteration
            k += 1

            if not np.isfinite(old_fval):
                break

            # Check if gnorm is already smaller than tolerance
            gnorm = vecnorm(gfk, ord=self.norm)
            if (gnorm <= self.gtol):
                break

            # Calculate rhok factor for Hessian approximation matrix update
            try:
                rhok = 1.0 / (np.dot(yk, sk))
            except ZeroDivisionError:
                rhok = 1000.0
            if np.isinf(rhok):  # this is patch for NumPy
                rhok = 1000.0

            # Hessian approximation matrix Hk (Bk in papers) update
            A1 = I - sk[:, np.newaxis] * yk[np.newaxis, :] * rhok
            A2 = I - yk[:, np.newaxis] * sk[np.newaxis, :] * rhok
            Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
                                                     sk[np.newaxis, :])

        # Store found fitness value in fval for result
        fval = old_fval

        # Create OptimizeResult object based on found point and value
        result = OptimizeResult(fun=fval, jac=gfk, hess_inv=Hk, nfev=sf.nfev,
                        njev=sf.ngev, x=xk,
                        nit=k)

        # Store in x_opt and f_opt
        x_opt = result.x
        f_opt = result.fun

        if retall:
            result['allvecs'] = allvecs

        return x_opt, f_opt

69:9: E741 ambiguous variable name 'I'
78:80: E501 line too long (82 > 79 characters)
80:31: E128 continuation line under-indented for visual indent
98:80: E501 line too long (90 > 79 characters)
99:80: E501 line too long (91 > 79 characters)
104:80: E501 line too long (90 > 79 characters)
141:54: E127 continuation line over-indented for visual indent
148:25: E128 continuation line under-indented for visual indent
149:25: E128 continuation line under-indented for visual indent


In [5]:
from IOHexperimenter import IOHexperimenter
fIDs = []
for fid in range(1, 25):
    fIDs.append(fid)
dIDs = [2, 3, 5, 10, 20, 40]
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs, [1], dIDs, 15)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name = "BFGS/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([BFGS(10000)])
#create a ZIP file for IOHprofiler upload
make_archive(path + experiment_name,"zip", path + experiment_name)

Running single-threaded
1 9 1.3040773855381953e-12
1 12 1.7301306353600522e-12
1 12 6.011572737905584e-13
1 12 6.385221739562411e-12
1 12 3.4567047188511096e-12
1 9 4.879913972889621e-12
1 9 5.447143614899839e-13
1 9 8.151724797055144e-13
1 12 4.4519125466613877e-13
1 12 1.4023796888111649e-11
1 12 2.560574720469551e-11
1 12 9.417791584166485e-13
1 12 1.8921832477328614e-13
1 12 1.485966496996119e-12
1 12 3.570118321360181e-11
1 16 5.435093144771289e-13
1 16 1.7267832315794692e-11
1 16 1.4867806902143316e-11
1 16 7.59457840429606e-12
1 16 3.717887070151912e-11
1 16 1.3185073489370312e-11
1 16 3.1462498200941143e-12
1 16 9.087116347171424e-11
1 16 6.825896852335935e-12
1 12 7.209174140921364e-12
1 16 1.1107472896234903e-11
1 12 3.030711649703749e-13
1 12 1.0395709066847843e-12
1 16 1.853243547934721e-11
1 16 5.107483374499363e-13
1 30 4.939187140759634e-10
1 24 3.796606301661184e-11
1 24 5.2336351074427085e-11
1 24 6.887641661059373e-11
1 24 8.617738138319345e-11
1 24 1.4767790524806622

<ipython-input-4-2a0955b6b2de>:131: RuntimeWarning: divide by zero encountered in double_scalars
  rhok = 1.0 / (np.dot(yk, sk))


17 2478 73.74269615271085
17 2774 30.07548652580243
17 3245 13.861823511485053
17 1795 11.113176670124258
17 2238 96.65591466710953
17 2552 14.04236076678755
17 2534 21.177277941665427
17 2426 97.60223753977853
17 3539 13.253140213189953
17 2121 17.65830139055343
17 13467 14.784442770859968
17 5833 35.67640531150838
17 9116 55.147848041907814
17 14719 10.783849941304213
17 5866 47.71107069260542
17 11234 24.43067894832983
17 10917 14.866628419571017
17 14401 14.730737149497726
17 4592 115.20851568839686
17 9203 36.960431521530786
17 9602 8.608307877241053
17 6970 10.901381121036547
17 12639 8.087196992311618
17 8950 33.65939406903527
17 10304 36.298663007217854
18 132 337.8293276507442
18 126 6.299099249489583
18 161 827.0115199153817
18 159 88.33482517519
18 122 305.9809253844994
18 165 127.9581135501045
18 110 51.41943339192082
18 167 14.045249785139669
18 150 1226.2738006449692
18 162 1226.2738006874292
18 147 492.79617414051967
18 122 11.629541330220679
18 116 23.78771806071042
18 

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/BFGS/experiment6-10-084555.zip'

2:17: E211 whitespace before '('
2:20: E231 missing whitespace after ','
6:1: E402 module level import not at top of file
8:25: E231 missing whitespace after ','
9:80: E501 line too long (101 > 79 characters)
11:80: E501 line too long (135 > 79 characters)
15:1: E265 block comment should start with '# '
16:36: E231 missing whitespace after ','
